In [1]:
import numpy as np

navs = ('No navegable', 'Navegable')

x = np.load('processed_nav_data.npy')
y = np.load('nav_data_labels.npy')

x_train, x_val, x_test = np.split(
    np.array(x), [int(len(x)*0.7), int(len(x)*0.85)])
y_train, y_val, y_test = np.split(
    np.array(y), [int(len(y)*0.7), int(len(y)*0.85)])

n_clases = len(navs)
n_features = len(x[0])

print(np.unique(y, return_counts=True))
print(len(x))
print(len(y))
print(n_clases)
print(n_features)


(array([0, 1]), array([361, 759]))
1120
1120
2
10


In [2]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(y),
                                                  y)
kf = KFold(n_splits=5, shuffle=True)
acc = 0
recall = np.array([0., 0.])
iteration = 1
for train_index, test_index in kf.split(x):
    print(f'Iteration {iteration}')
    iteration += 1
    x_train = x[train_index, :]
    y_train = y[train_index]

    clf = Sequential()
    clf.add(Dense(16, input_dim=n_features, activation='relu'))
    clf.add(Dense(16, activation='relu'))
    clf.add(Dense(1, activation='sigmoid'))
    clf.compile(loss='binary_crossentropy', optimizer='adam')
    clf.fit(x_train, y_train, epochs=50, batch_size=5, class_weight=(
        dict(zip(np.unique(y), class_weights))), verbose=0)

    x_test = x[test_index, :]
    y_test = y[test_index]
    y_pred = (clf.predict(x_test) > 0.5).astype("int32")
    cm = confusion_matrix(y_test, y_pred)
    acc += (cm[0, 0]+cm[1, 1])/len(y_test)
    recall[0] += cm[0, 0]/(cm[0, 0] + cm[0, 1])
    recall[1] += cm[1, 1]/(cm[1, 0] + cm[1, 1])
acc = acc/5
print('ACC = ', acc)
recall = recall/5
print('RECALL = ', recall)


2021-11-13 20:50:19.662349: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
